In [1]:

#@title #####  Setup

import sys, os , time, re, json, pytz, IPython, requests, codecs
import ipywidgets as widgets
from typing import List
from google.colab import drive
from IPython.display import display
from datetime import datetime 

tz = pytz.timezone('Turkey')

Drive = False #@param {type:"boolean"}
FileManager = False #@param {type:"boolean"}
Ftp = False #@param {type:"boolean"}
MySQL = False #@param {type:"boolean"}
XML = True #@param {type:"boolean"}
CSV = True #@param {type:"boolean"}

get = requests.get
head = requests.head

if Drive and not os.path.exists("/content/drive"):
  drive.mount('/content/drive')

if Ftp and not 'ftputil' in globals():
  !pip install ftputil &> /dev/null
  !sudo apt-get install lftp &> /dev/null
  import ftputil

if MySQL and not 'MySQLdb' in globals():
  !curl emasri.com &> /dev/null
  !pip install mysqlclient &> /dev/null
  init = True


SD_Main = "XS" #@param {type: "string"}
movieslib = "MV"
moviesbackup = "MVB"
tvlib = "TV"
tvbackup = "TVB"
musiclib = "MC"
musicbackup = "MCB"
websitedata = "WS"
websitebackup = "WSB"
generallib = "XT"
generalbackup = "XTB"

log_local = '/content/td_logs'
log_file = "/content/td_logs/td_log.txt"
log_drv = '/content/drive/MyDrive/General/td_logs'
log_info = "Info"
log_warning = "Warning"
log_error = "Error"
main = "Main"

def append_lines(filename,  line = '', lines = None):
  if lines or line:
    with open(filename, "a+") as file:
      file.seek(0)
      data = file.read(100)
      if len(data) > 0:
        file.write("\n")
        
      if line:
        file.write(line + "\n")

      if lines:
        for i in range(len(lines)):
          file.write(lines[i])
          if i < len(lines) - 1:
            file.write("\n")
    return True
  return False 

if not 'init' in globals():
  !rm -r "/content/sample_data"
  if os.path.exists("/content/drive"):
    now = datetime.now(tz).strftime("%y-%m-%d-%H-%M-%S")
    if SD_Main and os.path.exists(f'/content/drive/Shareddrives/{SD_Main}') :
      log_drv = f'/content/drive/Shareddrives/{SD_Main}/Downloads/td_logs'
      log_drv_file =  f'/content/drive/Shareddrives/{SD_Main}/Downloads/td_logs/td_log_{now}.txt'
      os.makedirs(log_drv, exist_ok= True)
    else:
      log_drv_file = f'{log_drv}/td_log_{now}.txt'
  else:
    log_drv = log_drv_file = ''
  init = True

os.makedirs(log_local, exist_ok = True)
if os.path.exists('/content/drive'):
  os.makedirs(log_drv, exist_ok = True)

class Logger:
  _instance = None
  @staticmethod
  def getInstance():
    if Logger._instance == None:
        Logger()
    return Logger._instance

  def __init__(self):
    if Logger._instance != None:
        raise Exception("Logger is a singleton.")
    else:
        Logger._instance = self
  
  logs = []
  def init(self, hold = False):
    if hold:
      self.logs.append(self.makelog('Log started.', log_info, main))
    else:
      self.log( 'Log started.', log_info,  main)

  def makelog(self, text: str, log_type: str = 'Info', source = main) -> str:
    now = datetime.now(tz).strftime("%y:%m:%d:%H:%M:%S")
    source = source.replace('_', ' ').title()
    start = "{} {:^7} {}:". format(now, log_type, source)
    text = text.replace("\n", f"\n{start} ")
    lg = f"{start} {text}"
    return lg
  
  def queue(self, text: str, log_type: str = 'Info', source = main):
    self.logs.append(self.makelog(text, log_type = log_type, source = source))

  def info(self, text: str = "", lines = None):
    src = sys._getframe(1).f_code.co_name
    if src == '<module>':
      src = main

    if lines:
      for line in lines:
        self.queue(line, log_info, src)
    if text:
      self.queue(text, log_info, src)

  def warning(self, text: str = "", lines = None):
    src = sys._getframe(1).f_code.co_name
    if src == '<module>':
      src = main

    if lines:
      for line in lines:
        self.queue(line, log_warning, src)
    if text:
      self.queue(text, log_warning, src)  

  def error(self, text: str = "", lines = None):
    src = sys._getframe(1).f_code.co_name
    if src == '<module>':
      src = main

    if lines:
      for line in lines:
        self.queue(line, log_error, src)
    if text:
      self.queue(text, log_error, src)
  
  def release(self):
    if (self.logs):
      if self.log(logs = self.logs):
        self.logs.clear()
      else:
        print("Logger: An error happend while trying to write log. logs are preserved.")

  def log(self, text = '', log_type= log_info,  source = main, logs = None):
    if not logs and not text:
      return True
    res = False
    if text:
      text = self.makelog(text, log_type, source)
    res = append_lines(filename = log_file, line = text, lines = logs)

    if res:
      self.save_log()
      return True
    else:
      print("Logger: log: An error happened while trying to write log.")
      return False

  def save_log(self):
    if log_drv and os.path.exists(log_drv) and os.path.isfile(log_file):
      !rsync -I "$log_file" "$log_drv_file"

logger = Logger.getInstance()   
info = logger.info
error = logger.error
warning = logger.warning
release = logger.release

if not os.path.isfile(log_file):
  logger.init(True)

if os.path.exists("/content/drive"):
  info("Google drive is mounted.")

nowstr = lambda separete = False: datetime.now(tz).strftime("%y-%m-%d-%H-%M-%S") if separete else str(round(time.time()))

skip = lambda c: True if c == "" or c == "s" else False
def ext (c):
  c = c.lower()
  return True if c == " " or c == "q" else False

def no(c):
  c = c.lower()
  return True if c == "n" else False

def getattrs(obj):
  attrs_ = []
  for attr in dir(obj):
    if not attr.startswith("__"):
      attrs_.append((attr, getattr(obj, attr)))
  return attrs_

def dump(obj, padding = True, filter = True):
  pad = "              " if padding else ""
  for attr in dir(obj):
    if filter and attr.startswith("__"):
      continue
    print("%s%s =  %r" % (pad, attr, getattr(obj, attr)))

def getattrstr(obj, filter = True, filter_empty = False, add_line = False):
  attrstr = ''
  pre = '\n' if add_line else ', '
  i = True
  for attr in dir(obj):
    if filter and attr.startswith("__"):
      continue
    val = getattr(obj, attr)
    if filter_empty and not val:
      continue
    if i :
      attrstr += "%s =  %r" % (attr, val)
      i = False
    else:
      attrstr += "%s%s =  %r" % (pre, attr, getattr(obj, attr))
  return attrstr
  
def list_lines(lines, str = False):
  if lines:
    if str:
      rtn = ''
      for r in lines:
        if r:
          rtn += (r +'\n')
      return rtn
    if not str:  
      for line in lines:
        print(line)

ext_re = r"\.\w+$"
extdot_re = r"\.(?=\w+$)"
file_re = r"^.*\.[A-Z,a-z,0-9]+$"
sd_re = r"(\/content\/drive\/Shareddrives\/)[A-Za-z0-9]+(?=\/)"
anybk_re = r"(?<=Shareddrives\/)([A-Za-z0-9]*(?=[Bb][1-9])|[A-Za-z0-9]*)"
backup_re = r"(?<=Shareddrives\/).*(?=\/)"
url_re = r"((http[s]?):\/\/)?([^:\/\s]+)((\/\w+\/)*\/)([\w\-\.]+[^#?\s]+)(.*)?(#[\w\-]+)?"
file_url_re = r"((http[s]?):\/\/)?([^:\/\s\"<>(\\n)]+)((\/\w{3}\/)*\/)([\w\-\.]+[^#?\s\"<>(\\n)]+)([^:\/\s\"<>(\\n)].*)?(#[\w\-]+)?"

def getbackup(path: str, bakid = "", bak_index = 0):
  if bakid:
    search = re.search(backup_re, path)
    if not search:
      error("Invalid shared drive path.")
      release()
      raise "Invalid shared drive path."
    return re.sub(backup_re, bakid, path, 1)
  elif bak_index:
    search = re.search(anybk_re, path)
    if not search:
      error("Invalid shared drive path.")
      release()
      raise "Invalid shared drive path."
    bakbase = search.group()
    replace = bakbase + f"B{bak_index}"
    bakpath = path.replace(bakbase, replace, 1)
    return (replace, bakpath)
  else:
    error("No backup id or index.")
    release()
    raise("No backup id or index.")

def pathsize(path, string = False):
  size = 0
  if not os.path.exists(path):
    return 0
  if os.path.isfile(path):
    size = os.path.getsize(path)
  else:  
    for path, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(path, f)
            size += os.path.getsize(fp)
  if string:
    return data_str(size)
  else:
    return size

def has_size(path, only_check = True):
  size = 0
  if not os.path.exists(path):
    return 0
  if os.path.isfile(path):
    size = os.path.getsize(path)
  else:  
    for path, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(path, f)
            size += os.path.getsize(fp)
            if only_check and size >= size_limit:
              return size
  if size >= size_limit:
    return size
  else:
    return 0

def is_hidden(path: str):
  return True if path[0] == '.' else False

def filter_checks(dirs):
  for dir in dirs:
    if dir.endswith('ipynb_checkpoints'):
      dirs.remove(dir)
  return dirs

def print_files(drs, range_start , range_end):
  for i in range(range_start, range_end):
    print("   {0}-  {1}".format(i+1, drs[i])) 

def transstring(name): 
  if not name:
    error('String is empty.')
    raise Exception('String is empty.')
  table = name.maketrans("",""," ,.;:!?'&#$%`!@^*_-/\\|'\"<>())[]{}")
  return name.translate(table).lower()

def data_str(byte_val):
  size = ByteToGB(byte_val)
  if (size >= 1) :
    return "{:.1f} GB".format(size)
  size = ByteToMB(byte_val)
  if (size >= 1) :
    return "{:.1f} MB".format(size)
  size = ByteToKB(byte_val)
  if (size >= 1) :
    return "{:.1f} KB".format(size)
  return "{:.1f} B".format(size)

def speed_str(byte_val):
  size_mb = ByteToMB(byte_val)
  if (size_mb >= 1) :
    return "{:.1f} MB".format(size_mb)
  else:
    return "{:.1f} KB".format(ByteToKB(byte_val))
    
def RoundTo1(n) :
  if n < 1 and (n - 0.9765625) >= 0 :
    return 1
  else: 
    return n

ByteToGB = lambda n: RoundTo1(n / 1073741824)
ByteToMB = lambda n: RoundTo1(n/ 1048576)
ByteToKB = lambda n: RoundTo1(n/ 1024)
contains = lambda string, search: search.lower() in string.lower()
time_str = lambda sec: time.strftime('%H:%M:%S', time.gmtime(sec))
size_limit = 40000

checkpoint = '.ipynb_checkpoints'
sharedbase = '/content/drive/Shareddrives/'

def check_ok(url, v = False):
  res = head(url).status_code
  if res != 200:
    print(f"  Resource not found {res} {url}")
    return
  print(f"  Ok 200.")

def write_json( dct, filename = 'scrape_output.json'):
  if not dct:
    error("Empty input")
    release()
    raise("Empty input")
  
  text = json.dumps(dct, ensure_ascii = False, indent = 2)
  output_file = filename
  i = 0
  while True:
    i += 1
    if os.path.exists(output_file):
      sch = re.search(extdot_re, output_file)
      if sch:
        output_file = re.sub(extdot_re, f"{i}.", filename)
      else:
        output_file = f"{filename}{i}"  
    else: 
      break

  with open(output_file, 'w', encoding = 'utf8') as file:
    file.write(text)

def read_json(filename, encoding = 'utf8'):
  with codecs.open(filename, 'r', encoding) as file:
    json_ = json.load(file)    
    return json_
    

########################################################

## File Manager

if FileManager:
  def mv(from_path, to_path, content = None, backup = False):
    if to_path:
      os.makedirs(to_path, exist_ok= True)  
      if content:
        info(f"Moving batch of files to: {to_path}")
        print("\n Enter selection to move [item1,item2,start-end  All: . Exit: q]:\n")
        print_files(content, 0 ,len(content))
        print()
        items: List[int] = []
        start = end = 0
        inpt = input()
        if ext(inpt):
          return
        if '.' == inpt:
          start = 0
          end = len(rng)      
        else:
          rngs = inpt.split(',')
          for rng in rngs:
            if '-' in rng:
              rngs = rng.split('-')
              start = int(rngs[0]) - 1
              end = int((rngs[1]))
              items.extend(range(start, end))      
              if end > len(content) or start < 0 :
                print("The ranges' ends are out of the directory file range.")
              continue
            else:
              i = int(rng)-1
              items.append(i)
      else:
        info(f'Moving: {from_path}\nProcess backup: {backup}')
        items = [0]

      for i in items:
        rename  = ""
        
        p1 = (from_path + "/" +  content[i]) if content else from_path
        p2 = to_path +  rename
        info(f"Moving {p1} to: {p2}")
        os.makedirs(to_path, exist_ok= True)
        !mv "$p1" "$p2"
        if backup:
          info(f"Processing backup.")
          for i in range(1,4):
            p11bak = getbackup(p1, bak_index = i)
            tpbak = getbackup(to_path, bak_index = i)
            if not (p11bak[0]in from_path and tpbak[0] in to_path):
              tp =  tpbak[1]
              p11 = p11bak[1]
              p21 = tp +  rename
              if os.path.exists(p11):
                os.makedirs(tp, exist_ok= True)
                print(f"Moving {p11} to {p21}\n")
                info(f"Moving {p11} to {p21}")
                !mv "$p11" "$p21"
              else:
                print(f"Backup {p11} Doesn't exist.\n")
                warning(f"Backup {p11} Doesn't exist.")

      print("\n   .....................\n           Done\n")
      info("Moving done.")
      return            
    else:
      error("No distination provided.")
      return

  def cp(from_path, to_path, content = None, backup = False, beforedate = None, afterdate = None,):
    if to_path:
      os.makedirs(to_path, exist_ok= True)  
      if content:
        info(f"Copying batch of files to: {to_path}")
        print("\n Enter selection to copy [item1,item2,start-end  All: . Exit: q]:\n")
        print_files(content, 0 ,len(content))
        print()
        items: List[int] = []
        start = end = 0
        inpt = input()
        if ext(inpt):
          return
        if '.' == inpt:
          start = 0
          end = len(rng)      
        else:
          rngs = inpt.split(',')
          for rng in rngs:
            if '-' in rng:
              rngs = rng.split('-')
              start = int(rngs[0]) - 1
              end = int((rngs[1]))
              items.extend(range(start, end))      
              if end > len(content) or start < 0 :
                print("The ranges' ends are out of the directory file range.")
              continue
            else:
              i = int(rng)-1
              items.append(i)
      else:
        info(f'Copying: {from_path}\nProcess backup: {backup}')
        items = [0]

      for i in items:
        newname  = ""

        p1 = from_path + "/" + content[i] if content else from_path
        if newname:
          p2 = from_path + "/" +  newname if content else os.path.dirname(from_path) + '/' + newname
          print("  Renaming:  {} to {}".format(items[i], newname))
          !mv "p1" "$p2"
          p1 = p2

        print(f"Copying:  {p1} to {to_path}")
        info(f"Copying:  {p1} to {to_path}")
        !rsync -r --size-only  "$p1" "$to_path"
        if backup:

          for i in range(1,4):
            p3bak = getbackup(to_path, bak_index = i)
            if not (p3bak[0]in from_path and p3bak[0] in to_path):
              p3 =  p3bak[1]
              os.makedirs(p3, exist_ok= True)
              print(f"Copying  {p1} to {p3}\n")
              info(f"Copying:  {p1} to {p3}")
              !rsync -r --size-only "$p1" "$p3" 

      print("\n   .....................\n           Done\n")
      info(f"Copying done.")
      return
    else:
      error("No distination provided.\n")
      return

  def rename(path, newpath, backup, content = None):
    print(f"Renaming:  {path[27:]} to {newpath[27:]}")
    !mv "$path" "$newpath" 
    if backup:
      for i in range(1,4):
        pathbu = getbackup(path, bak_index = i)
        newpathbu = getbackup(newpath, bak_index = i)
        if not pathbu[0] in path and os.path.exists(pathbu[1]):
          path = pathbu[1]
          newpath = newpathbu[1]    
          print(f"Renaming:  {path[27:]} to {newpath[27:]}")
          !mv "$path" "$newpath"      
      
    print("......................")
    print("       Done")
    return  
    
  def rm(delpath, process_backup = False, content = None, v = False):
    if content:
      content.sort()
      info(f'Deleing files in from: {delpath}\nProcess backup: {process_backup}')
      print(" Enter directory selection to remove: [s-e],2  '.' = All ")
      items: List[int] = []
      print_files(content, 0 ,len(content))
      print()
      start = end = 0
      inpt = input()
      if ext(inpt):
        return
      if '.' == inpt:
        start = 0
        end = len(rng)      
      else:
        rngs = inpt.split(',')
        for rng in rngs:
          if '-' in rng:
            rngs = rng.split('-')
            start = int(rngs[0]) - 1
            end = int((rngs[1]))
            items.extend(range(start, end))      
            if end > len(content) or start < 0 :
              print("The ranges' ends are out of the directory file range.")
            continue
          else:
            i = int(rng)-1
            items.append(i)

    elif delpath:
      info(f'Deleing: {delpath}\nProcess backup: {process_backup}')
      items = [0]
    else:
      return -1

    for i in items:
      path = delpath + "/" + content[i] if content else delpath
      if v:
        print(f"Deleting path:  {path}")
      info(f'Deleing path: {path}')
      if os.path.exists(path):
        rt = !rm -r "$path"
        if rt:
          error(lines = rt)
          list_lines(rt)
      else:
        error(f"{path} doesn't exist.")
        continue
      if process_backup:
        for i in range(1,4):
          pt = getbackup(path, bak_index = i)
          if not pt[0] in delpath and os.path.exists(pt[1]):
            pt = pt[1]
            if v:
              print(f"Deleting path: {pt}")
            info(f'Deleing file: {pt}')
            rt = !rm -r "$pt"
            if rt:
              error(lines = rt)
              list_lines(rt)
          else:
            warning(f"Backup path: {pt} doesn't exist.") 
    return

  def is_cp(name):
    return True if contains(checkpoint, name) else False

  def del_cp(path):
    p0 = f"{path}/{checkpoint}"
    if os.path.exists(p0):
      !rm -r "$p0"

  def dircontent(path):
    del_cp(path)
    return os.listdir(path)

  sizeDiffers = (1,'Size differs')
  doesntExist = (2,"Doesn't Exist")
  emptySizeLimit = 40000

  class Sync_Status:
    def __init__(self, path, src: int, message = '', name1 = '', name2 = '', size1 = '', size2 = '', status = doesntExist):
      self.path = path
      self.src = src
      self.message = message
      self.name1 = name1
      self.name2 = name2
      self.size1 = size1
      self.size2 = size2
      self.status = status

  def compare_dirs(first, second, sync = False, syncNames = False, delEmpty = False):
    info(f'Comparing sync status.\nFirst path: {first}\nSecond path: {second}')
    if first and os.path.exists(first) and second and os.path.exists(second):
      del_cp(first)
      del_cp(second)
      content1 = os.listdir(first)
      content2 = os.listdir(second)
      content1.sort()
      content2.sort()
      synced = []
      sync_list: List[Sync_Status] = []
      emty1 = emty2 = 0
      if content1:
        ln1 = len(content1)
        ln2 = len(content2)
        print("         {:<74}{:^43} {:^12}{:^12}{:^14}\n".format("In Dir 1", "In Dir 2", "Size in 1",  "Size in 2", 'Sync Status'))
        for item in content1:
          path1 = f"{first}/{item}"
          path2 = f"{second}/{item}"
          size1_raw= pathsize(path1)
          size1 = data_str(size1_raw)
          size2 = ""
          exists2 = False
          status2 = ''
          synced_ = 'Synced'
          unsynced_ = 'Unsynced'
          recheck = ''
          if delEmpty and size1_raw <= emptySizeLimit:
            !rm -r "$path1"
            ln1 -= 1
            info(f"Empty directory deleted {path1}")
            emty1 += 1
            size1 = 0

          if item in content2:        
            exists2 = True 
            status2 = "Exists"
            content2.remove(item)
            
          else:
            recheck = research_name(item, content2)
            if recheck:
              exists2 = True
              content2.remove(recheck)
              path2 = f"{second}/{recheck}"
              status2 = f'[{recheck}]'
              info(f"{item} exists as {recheck} in 2.")

          if exists2:
            size2_raw = pathsize(path2)
            if delEmpty and size2_raw <= emptySizeLimit:
              !rm -r "$path2"
              ln2 -= 1
              info(f"Empty directory deleted {path2}")
              emty2 += 1            
              continue

            if recheck and syncNames:
              info(f"Renaming {recheck} to {item}")
              p0 = f"{second}/{recheck}"
              p1 = f"{second}/{item}"
              ot = !mv "$p0" "$p1"
              if ot:
                error(ot)
              else:
                path2 = p1
                status2 = 'Renamed'
                info(f"Item {recheck} renamed to {item} in 2.")
            

            size2 = data_str(size2_raw)
            diff = False if abs(size1_raw - size2_raw) <= 50000 else True
            if not diff:
              msg = "    {:<80}{:^40}{:>12}{:>12}{:^20}".format(item, status2, size1, size2, synced_)
              synced.append(msg)
            else: 
              s_status = sizeDiffers[1]
              msg = "{:<80}{:^40}{:>12}{:>12}{:^20}".format(item, status2, size1, size2, s_status)
              if  size1_raw > size2_raw:
                status = Sync_Status(path= path1, src= 1, message= msg, status= sizeDiffers)
                info(f"An item added to the sync list 1: {path1}, size differs in path 2.")
              else:
                status = Sync_Status(path= path2, src= 2, message= msg, status= sizeDiffers)
                info(f"An item added to the sync list 2: {path2}, size differs in path 1.")
              sync_list.append(status)
          else:
            if not size1:
              continue
            status2 = ''
            s_status = doesntExist[1]
            msg = "{:<80}{:^40}{:>12}{:>12}{:^20}".format(item, status2, size1, size2, s_status )
            status = Sync_Status(path= path1, src= 1, message= msg, status= doesntExist)
            sync_list.append(status)
            info(f"An item added to the sync list 1: {path1}, doesn't exist in path 2.")
          
        if content2:
          for item in content2:                    
            path2 = f'{second}/{item}'
            size2_raw =  pathsize(path2, False)
            if delEmpty and size2_raw <= emptySizeLimit:
              !rm -r "$path2"
              info(f"Empty directory deleted {path2}")
              emty2 += 1
              ln2 -= 1
              continue          
            size2 = data_str(size2_raw)
            msg = "{:<80}{:^40}{:>12}{:>12}{:^20}".format("", item, '', size2, "Doesn't Exist")
            status = Sync_Status(path= path2, src= 2, message= msg, status= doesntExist)
            sync_list.append(status)
            info(f"An item added to the sync list 2: {path2}, doesn't exist in path 1.")

        for item in synced:
          print(item)
          info(item)

        if sync_list:
          sync_list.sort(key= lambda obj: obj.status[0])
          for i in range(len(sync_list)):
            item = sync_list[i]
            msg = f"{str(i + 1)+' - ' if sync else '    '}{item.message}"
            print(msg)
            info(item.message)
                
        msg = f"\nTotal items in path 1: {ln1}   Total items in path 2:  {ln2}   Unsynced items: {len(sync_list)}.\n"
        msg += f"{f'Deleted empty folders in path 1:  {emty1}  ' if emty1 else ''}{f'Deleted empty folders in path 2:  {emty2}' if emty2 else ''}\n"
        info(msg)
        print(msg)
        
        if sync and sync_list:
          print('Enter a selection for the synchronization:  e.g. i1, i2, i1-i5\n')
          inp = input().strip()
          if (ext(inp) or no(inp)):
            return
          rngs = []  
          if inp == '.':
            rngs.extend(range(len(sync_list)))
          else:
            selects = inp.split(',')
            for select in selects:
              try:          
                if '-' in select:
                    splt = select.split('-')
                    s = int(splt[0]) - 1
                    e = int(splt[1])
                    rngs.extend(range(s, e))
                else:
                  rngs.append(int(select) - 1)
              except Exception as e:
                error(str(e))
                continue
          msg = f"Syncing {first} with {second}.\nTotal copying items: {len(rngs)}"
          info(msg)
          print(msg)
          for i in rngs:
            p0 = sync_list[i].path
            p1 = second if sync_list[i].src == 1 else first
            print(f"Copying: {os.path.basename(p0)} to {p1}")
            info(f"Copying: {os.path.basename(p0)} to {p1}")
            !rsync --size-only -P -h -r "$p0" "$p1"
        print("\n.............\n    Done\n")
    else:
      msg = f"{first} {'exists' if os.path.exists(first) else 'does not exist'}\n{second} {'exists' if os.path.exists(second) else 'does not exist'}"
      error(msg)

  def research_name(key, content2):
    key = key.strip()
    if key:
      key = os.path.splitext(key)[0]
      keytrans = transstring(key)
      ln = len(keytrans)
      if ln >= 15:
        for i in range(len(content2)):
          trans = transstring(os.path.splitext(content2[i])[0])
          ln1 = len(trans)
          if ln1 < 15:
            continue
          if ln1 <= ln:
            n0 = trans  
            n1 = keytrans
          else:
            n0 = keytrans
            n1 = trans
          if contains(n1, n0):
            return content2[i]
    return False

  def mt(path, bd = None, ad = None, backup = False, prompt = False):
    if not (bd or ad):
      return
    info(f'Deleing files dated before {datetime.fromtimestamp(bd).strftime("%Y-%m-%d")} in {path}')
    files = next(os.walk(path))[2]
    check = bd and ad
    filter = []
    for file in files:
      p0 = os.path.join(path, file)
      md = os.path.getmtime(p0)
      if (check and md <= bd and md >= ad) or (bd and md <= bd) or (ad and md >= ad):
        dstr = datetime.fromtimestamp(md).strftime('%Y-%m-%d')
        size = data_str(os.path.getsize(p0))
        msg = f'{dstr} - {size} {file}'
        info(msg)
        print(msg)
        if prompt:
          filter.append(file)
          continue         
        info(f"Deleting {p0}..")
        rm(p0, backup, False)
    if prompt:
      rm(delpath= path, process_backup= backup, content= filter)
    print('..... Done .....\n')

  def check_sd_base(path):
    sch = re.search(sd_re, path)
    if sch:
      p0 = sch.group()  
      if os.path.exists(p0):
        return p0
    return False

  def makedirs(path, backup = False):
    if path:
      info(f"Creating dir: {path}")
      os.makedirs(path, exist_ok = True)
    if backup:
      for i in range(1, 4):
        nd = getbackup(path, bak_index = i)
        np = nd[1]
        check = check_sd_base(np)
        if check:
          info(f"Creating backup dir: {np}")
          os.makedirs(nd, exist_ok = True)
        else:
          warning(f"{nd[0]} doesn't exist.")

########################################################

def slist(valuelist, sep = ', ', wrap = False):
  if valuelist:
    valuestr = ""
    i = 0
    for value in valuelist:
      i += 1
      if i == len(valuelist):
        valuestr += value.strip()
      else:  
        valuestr += value.strip() + sep
    return f'"{valuestr}"' if wrap else valuestr

if CSV:
  import csv  
  def wcsv(header, data, file, quoting = csv.QUOTE_MINIMAL):
    if data:
      islist = isinstance(data, list)
      if islist:
        datadict = isinstance(data[0], dict)
      else:
        datadict = isinstance(data, dict)

      with open(file, 'w', newline="") as csvfile:
        if  datadict:
          writer = csv.DictWriter(csvfile, fieldnames = header, quoting=csv.QUOTE_MINIMAL, escapechar=' ')
          writer.writeheader()
        else:
          writer = csv.writer(csvfile, quoting = quoting, escapechar=' ')
          writer.writerow(header)
        if islist:  
          writer.writerows(data)
        else :
          writer.writerow(data)
    else:
      print("wcsv error: data is empty.")

if XML:
  import ast
  from bs4 import BeautifulSoup
  itext = lambda item, tag = "":(item.get(tag) if tag else item.text) if item else ""

  def xmltest(items):
    if not items:
      print("No items")
      return
    else:
      print(f"Total items: {len(items)}\nSample:\n {items[round(len(items)/2)]}")

  valtl = "x"
  elre = r"(?<=<>).*(?=<\/>)"

  def xmlview(file, element, attr = ""):
    items = xmlcall(file, element)
    print(f"All items: {len(items)}\n")
    for i in range(len(items)):
      val = items[i].text if not attr else items[i].get(attr)
      print(f"Item {i:<5}->   {val:<10}")
    return items

  def xmledit(file, element, value= None, numbers = False, savefile = None):
    if os.path.isfile(file):
      if not savefile:
        savefile = file
      count = subs = 0
      with open(file, 'r') as xfile:
        target = r'(?<=<' + element + r'>).*(?=<\/'+ element + '>)'
        txt = xfile.read();

        if value:
          newtxt = txt
          print(f"{'Updating values in:':>19} <{element}>\n{'Reevaluation:':>19} {value} (x: old value)\n")
        else:
          print(f"Values in <{element}>:\n")

        mchs = re.findall(target, txt)
        if mchs:
          matches = set(mchs)
          count = len(matches)
          i = 0
          for mch in matches:
            if value:
              oldval = mch if not numbers else ast.literal_eval(mch)
              newval = eval(value, {"x": oldval})
              thistarget = target.replace(".*", mch)
              sub = re.subn(thistarget, str(newval), newtxt)
              newtxt = sub[0]
              subs += sub[1]
              print( f"{i:<4}- {str(mch):<15} {'->':<15} {str(newval):<15}Subs {sub[1]}")
            else:
              print(f"{i:<4}- {mch:<10}")
            i += 1
        if value:
          with open(savefile, 'w') as wfile:
            wfile.write(newtxt)
          print(f"\nTotal matches: {count}     Values changed: {subs}\n")
        else:
          print(f"\nTotal values: {count}\n")

  def xmlcall(file, element, callback = None):
    if os.path.isfile(file):
      with open(file, 'r') as f:
          data = f.read()
    else:
      print("File not there.")
      return
    xml_data = BeautifulSoup(data, "xml")
    items = xml_data.find_all(element)
    return callback(items) if callback else items


In [ ]:

#@title ##### Scraping

from bs4 import BeautifulSoup
import requests
import json
import time

Base = "" #@param {type:"string"}
Target = "" #@param {type:"string"}
Limit = 0 #@param {type:"number"}

def scrape_posts(link_base, limit = 0, target_class = '', id_var = ''):
  limit = 100 if limit == 0 else limit
  var = f"?{id_var}=" if id_var else ''
  scrape = {}
  posts = []
  for i in range(limit + 1):
    time.sleep(0.1)
    url = f"{link_base}/?{var}{i}"
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      page = soup.find('div', {"class": target_class})
      if page:
        print(f'Post found: {url}')
        posts.append(str(page))
        continue
    print(f'No Post:   {url}')

  if posts:
    scrape['posts'] = posts
    return scrape
  else:
    print('No posts found.')
    return ''

json_obj = scrape_posts(Base, target_class = Target, limit = Limit)
write_json(json_obj)

In [ ]:

#@title ##### Json

import os, json
#@markdown ##### <b>&nbsp;&nbsp;&nbsp;Target:&nbsp;&nbsp;&nbsp;json_obj<b>
Read = 100
Write = 200

Operation = Write #@param ["Read", "Write"] {type:"raw"}
File = "/content/newfile-img-migrated.json" #@param {type:"string"}
Encoding = 'utf-8-sig' #@param ['utf-8-sig', 'utf8'] {type:"string"}


if Operation == Read and File:
  json_obj = read_json(File, Encoding)
else:
  Source = json_obj #@param {type:"raw"}
  if File: 
    write_json(Source, File)
  else:
    write_json(Source)


In [ ]:

#@title ##### Scan Json

Source = json_obj #@param {type:"raw"}
Root = "posts" #@param {type: "string"}
InnerKey = "thumb" #@param {type: "string"}
Expression = None #@param {type: "raw"}
Substitute = "" #@param {type: "string"}
Limit = 0 #@param {type: "raw"}
Callback = check_ok #@param ["None"] {type:"raw", allow-input: true}
Print = True #@param {type:"boolean"}

def scan_json(scan_obj, limit = 0 , root = "" , key = "", exp = "", 
  replace = "", callback = None, v = False):
  toscan = scan_obj[root] if root else scan_obj
  if type(toscan) == str:
    toscan = [toscan]
    
  if type(limit) == str:
    if '-' in limit:
      splt = limit.split('-')
      s = int(splt[0])
      e = int(splt[1])
    else:
      s = 0
      e = int(limit)        
  elif limit:
    s = 0
    e = limit
  else:
    s = 0
    e = len(toscan)

  if replace and exp:
    print(f"Substituting \"{exp}\" with \"{replace}\"")
    for i in range(s, e):
      print(f"\nPost {i}")
      element = toscan[i][key] if key else toscan[i] 
      scaned = re.subn(exp, replace, element)
      if scaned:
        print(f"Substitutions: {scaned[1]}")
        if key:
          toscan[i][key] = scaned[0]
        else:
          toscan[i] = scaned[0]
      else:
        print(f"No subtitutions.")
    return

  for i in range(s, e):
    print(f"Item {i}")
    element = toscan[i][key] if key else toscan[i] 
    mchs = re.findall(exp, element) if exp else [element]
    if mchs:
      if callback:
        for mch in mchs:
          if v:
            print(f" {mch}")
            try:
              callback(mch)
            except (Exception, KeyboardInterrupt) as e:
              print(str(e))
      else:
        for item in mchs:
          print(item)          
    else:
      print(" No matches.")
    print()
  

if Source:
  scan_json(Source, Limit, Root, InnerKey, Expression, Substitute, Callback, Print)


In [ ]:

#@title ##### Check Json Item

obj = json_obj['posts'][103] 
obj

In [ ]:

#@title ##### Scrape Item

from bs4 import BeautifulSoup

soup = BeautifulSoup(t, 'html.parser')
target_class = 'browse-text'
el = soup.find('div', {"class": target_class})

In [ ]:

#@title ##### Scan Item

hashtags = set()
hashtag_re = r"#[^\s<>\/]*(?=\s|<|>)"
mchs = re.findall(hashtag_re, str(el), re.MULTILINE)
if mchs:
  for m in mchs:
    hashtags.add(m)

for m in hashtags:
  print(m)

str(el)

In [ ]:


#@title ##### FTP

import re
Dir = "/etc" #@param {type:"string"}
Size = False

FTP_Host = "emasri.com" #@param {type: "string"}
FTP_Username = " emasri" #@param {type: "string"}
FTP_Password = "  " #@param {type: "string"}

FTP_Host = FTP_Host.strip()
FTP_Username = FTP_Username.strip()
FTP_Password = FTP_Password.strip()

def ls(dir = "", size = False):
  total_size = 0
  items = []
  with ftputil.FTPHost(FTP_Host, FTP_Username, FTP_Password) as ftp:
    if not dir:
      dir = ftp.curdir
    path, dirs, files = next(ftp.walk(dir))
    files.sort()
    dirs.sort()

    for name in dirs:
      item = f"{path}/{name}"
      items.append(item) 

    for name in files:
      item = f"{path}/{name}"
      items.append(item)
    
    for i in range(len(items)):
      print(f"{str(i+1).ljust(3)} - {items[i]}")
      
    if total_size:
      print(f"\nTotal size: {data_str(total_size)}")

ls(Dir)

In [ ]:

#@title ##### Transfer

Upload = 0
Download = 1

Path = "/content/td_logs/a file.txt" #@param {type: "string"}
SavePath = '/etc' #@param {type: "string"}
Action = Upload #@param ["Upload", "Download"] {type: "raw"}
done_path = '/content/done'
os.makedirs(done_path, exist_ok = True)
os.makedirs(SavePath, exist_ok = True)
total_size = 0
total_str = ""
done = 0
patch_size = 0

layout_15 ={'width':'auto', 'flex':'15 1 0%', 'align_items':"center"}
layout_10 ={'width':'auto', 'flex':'10 1 0%', 'align_items':"center"}
layout_8 ={'width':'auto', 'flex':'8 1 0%', 'align_items':"center"}
layout_7 ={'width':'auto', 'flex':'7 1 0%', 'align_items':"center"}
layout_5 ={'width':'auto', 'flex':'5 1 0%', 'align_items':"center"}
layout_4 ={'width':'auto', 'flex':'4 1 0%', 'align_items':"center"}
layout_3 ={'width':'auto', 'flex':'3 1 0%', 'align_items':"center"}
layout_2 ={'width':'auto', 'flex':'2 1 0%', 'align_items':"center"}
layout_1 ={'width':'auto', 'flex':'1 1 0%', 'align_items':"center"}
layout_r = {'padding-left':'100px','flex_flow':'row','align_items':'flex-start','width':'95%','justify_content':'space-around'}
style0= {"padding-left":"200px","padding-right":"200px"}

empty_label_1 = widgets.Label(value = "", layout = layout_1)
empty_label_2 = widgets.Label(value = "", layout = layout_2)
empty_label_3 = widgets.Label(value = "", layout = layout_3)
empty_label_4 = widgets.Label(value = "", layout = layout_4)
empty_label_5 = widgets.Label(value = "", layout = layout_5)

progress_bar = widgets.FloatProgress(
      value=0,
      min=0,
      max=100,
      step=0.1,
      bar_style='info',
      orientation='horizontal',
      layout=layout_15)

done_label = widgets.Label(
    value = f"",
    layout = layout_2
)

progress_label = widgets.Label(
    value = f"0%",
    layout = layout_1
)

download_label = widgets.Label(
    value = f"Downloading",
    layout = layout_r
)

r1 = widgets.Box([ empty_label_2, done_label, empty_label_1, progress_label, progress_bar, empty_label_2], layout = layout_r, style = style0)

def update_upload(chunk):
  global done, total_size, progress_bar, progress_label, done_label
  done += len(chunk)
  progress = round(done*100/total_size, 1)
  progress_bar.value = progress
  if progress == 100:
    progress_bar.bar_style = 'success'
  done_label.value = f"{data_str(done)} {total_str}"
  progress_label.value = f"{progress}%"

def update_download(chunk):
  global done, download_label, patch_size 
  patch = len(chunk)
  done += patch
  result = ' -  Transfered Successfully' if patch < patch_size else ''
  download_label.value = f"Transfered {data_str(done)} {result}"
  if not patch_size:
    patch_size = patch

def upload_dir(path, save, callback = None):
  !lftp ftp://ghiras@ghirasalkhaeer.com:m5NhW3TqgQfnW7K@ftp.emasri.com -e "set ftp:ssl-allow no; mirror -R $path $save ;"

def upload(path, save, callback = None):
  pass
  #!lftp ftp://ghiras@ghirasalkhaeer.com:m5NhW3TqgQfnW7K@ftp.emasri.com -e "set ftp:ssl-allow no; mirror -R $path $save ;"


def download(path, savepath, callback = None):
  global total_size, total_str
  total_size = 0
  total_str = ''
  with ftputil.FTPHost(FTP_Host, FTP_Username, FTP_Password) as ftp:
    target = f"{done_path}/{os.path.basename(path)}"
    ftp.download_if_newer(path, target, callback = callback)
    if has_size(target):
      !mv "$target" "$savepath"

if Path:
  if Action == Upload:
    display(r1)
    if os.path.isdir(Path):
      upload_dir(Path, SavePath)
    else: 
      upload(Path, SavePath, callback = update_upload)
  else:
    display(download_label)
    download(Path, SavePath, callback = update_download)


In [ ]:

#@title ##### Zip

import os

Zip = 10
Unzip = 20

Operation = Zip #@param ["Zip", "Unzip"] {type: "raw"}
Input = "" #@param {type: "string"}
Save = "" #@param {type: "string"}

def is_zip(name):
  sch = re.search(ext_re, name)
  if sch:
    res = sch.group()
    return res == '.zip' or res == '.gz' or res == '.rar' or res == '.7z'
  return False  

def zip(input, save: str = "", callback = None):
  save = save.strip()
  if input:
    if not save:
      save = f"zipped-files-{nowstr(True)}.zip"
    else:  
      if not (is_zip(save)):
        save += ".zip"
    print(f"Zipping: {input}\nTo: {save}")
    !zip -r "$save" $input
    if has_size(save):
      print(f'Done, check output: {save}')

def unzip(input, save: str = "", callback = None):
  save = save.strip()
  if input:
    print(f"Unzipping: {input}\nTo: {save}")
    saving = "-d " + f'"{save}"' if save else ""
    !unzip "$input" $saving
    print(f'Done, check output: {save}')

if input:
  if Operation == Zip:
    zip(Input, Save)
  else:
    unzip(Input, Save)  

In [ ]:

#@title ##### Files Manager

Copy = 100
Move = 200  
NewFolder = 300
Rename = 400

Delete =  500 
To_Replace = 501 

Compare = 600  
Sync = 601

Clean = 701 

Operation = Copy #@param ["Copy", "Move", "Rename", "Delete", "NewFolder", "Delete", "To_Replace", "Compare", "Sync"] {type:"raw"}

From_Path = "/content/drive/Shareddrives/MV/Movies" #@param ["", "/content/drive/Shareddrives/MV/Movies", "/content/drive/Shareddrives/MC/Music", "/content/drive/Shareddrives/TV/TV Shows", "/content/drive/Shareddrives/MVB1/Movies", "/content/drive/Shareddrives/MCB1/Music", "/content/drive/Shareddrives/TVB1/TV Shows", "/content/drive/Shareddrives/MVB2/Movies", "/content/drive/Shareddrives/TVB2/TV Shows", "/content/drive/Shareddrives/MCB2/Music", "/content/drive/Shareddrives/MVB3/Movies", "/content/drive/Shareddrives/MVB3/Music", "/content/drive/Shareddrives/TVB3/TV Shows", "/content/drive/MyDrive/Library/Movies", "/content/drive/MyDrive/Library/TV Shows", "/content/drive/MyDrive/Library/Music"] {allow-input: true}
To_Path = "/content/drive/Shareddrives/SM4/Library/Movies"  #@param ["", "/content/drive/Shareddrives/MV/Movies", "/content/drive/Shareddrives/MC/Music", "/content/drive/Shareddrives/TV/TV Shows", "/content/drive/Shareddrives/MVB1/Movies", "/content/drive/Shareddrives/MCB1/Music", "/content/drive/Shareddrives/TVB1/TV Shows", "/content/drive/Shareddrives/MVB2/Movies", "/content/drive/Shareddrives/TVB2/TV Shows", "/content/drive/Shareddrives/MCB2/Music", "/content/drive/Shareddrives/MVB3/Movies", "/content/drive/Shareddrives/MVB3/Music", "/content/drive/Shareddrives/TVB3/TV Shows", "/content/drive/MyDrive/Library/Movies", "/content/drive/MyDrive/Library/TV Shows", "/content/drive/MyDrive/Library/Music"] {allow-input: true}

Multi = False #@param {type:"boolean"}
Backup = False #@param {type:"boolean"}
Rename_ = False #@param {type:"boolean"}
Movie_Name = False #@param {type:"boolean"}
Show_Name = False #@param {type:"boolean"}
Delete_Empty = True #@param {type: "boolean"}

From_Path = From_Path.strip()
To_Path = To_Path.strip()

if Multi:
  dirs = dircontent(From_Path)
  try:
    if 1 < Operation/Delete < 1.1 :
      toreplace = True if Operation == To_Replace else False
      rm(From_Path, Backup, toreplace, content= dirs)

    elif From_Path and To_Path:
      if Operation == Copy:
         cp( from_path = From_Path, to_path = To_Path, backup = Backup, content= dirs)
      elif Operation ==Move:
        mv( from_path = From_Path, to_path = To_Path, backup = Backup, content= dirs)

       
  except KeyboardInterrupt as k:
    k

elif Operation == NewFolder:
  makedirs(From_Path, Backup)

elif  Operation == Compare:
  compare_dirs(From_Path, To_Path, False, syncNames= Rename_, delEmpty = Delete_Empty)

elif  Operation == Sync:
  compare_dirs(From_Path, To_Path, True, syncNames= Rename_, delEmpty = Delete_Empty)

elif Operation == Rename:
  rename(path= From_Path, newpath= To_Path, backup= Backup)

elif Operation == Delete and From_Path:
  print(f"Deleting: {From_Path}. Are you sure?                    Enter for confirm. ")
  inpt = input()
  if inpt == "" or inpt == "y":
    rm(From_Path, Backup, False)

elif Operation == To_Replace and From_Path:
    rm(From_Path, Backup, True)

elif From_Path and To_Path:
  if Operation == Move:
    mv(from_path= From_Path, to_path= To_Path, backup = Backup)
  elif Operation == Copy:
    cp(from_path= From_Path, to_path= To_Path, backup = Backup)

release()

In [ ]:

#@title ##### Check Old Files

from datetime import datetime
#/content/drive/Shareddrives/SM2/Library/TV Shows/Friends 1994/S01
Path = "" #@param {type:"string"}
Before_Date = "2022-09-01" #@param {type:"date"}
After_Date = "2022-08-10" #@param {type:"date"}

Process_Backup = True #@param {type:"boolean"}
Confirm_Delete = False #@param {type:"boolean"}

bd = datetime.strptime(Before_Date, '%Y-%m-%d').timestamp() if Before_Date else None
ad = datetime.strptime(After_Date, '%Y-%m-%d').timestamp() if After_Date else None
mt(Path, bd, ad, backup = Process_Backup, prompt = Confirm_Delete)
release()

In [ ]:
#@title ##### Decode Unicode

import unicodedata

posts = scrape["posts"]

for i in range(len(posts)):
  item = posts[i]
  decoded = unicodedata.normalize('NFKD', item)
  posts[i] = decoded
  
scrape["posts"] = posts

In [ ]:

#@title ##### URL

import urllib

Quote = 100
Unquote = 200
Operation = Unquote #@param ["Quote", "Unquote"] {type: "raw"}
String = "" #@param {type: "string"}

if Operation == Quote:
  string = urllib.parse.quote(String)
else:
  string = urllib.parse.unquote(String)

string

In [ ]:

#@title ##### MySQL

import MySQLdb

Ping = 10
Connect = 20
Query = 30
Callback = 40
CurlServer = 50

Operation = Callback #@param ["Ping", "Connect", "Query", "Callback", "CurlServer"] {type:"raw"}

MySQL_Host = "5.2.85.161" #@param {type:"string"}
MySQL_Username = "" #@param {type:"string"}
MySQL_Password = "" #@param {type:"string"}
MySQL_DB = "emasri_ghiras_prod" #@param {type:"string"}
Input = mysql_call #@param {type:"string"} {type:"raw"}

def mysql_ensure(conn, v = False):
  status = conn.stat()
  if status.startswith("Lost"):
    print('Reconnecting.')
    conn.ping(True)
    if v:
      print(conn.stat())
  elif v:
    print(status) 

def mysql_query(host, user, password, db, query):
  with MySQLdb.connect(host, user, password, db, connect_timeout = 3306) as conn:
    cur: MySQLdb.cursors.Cursor = conn.cursor()
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    if res:
      for item in res:
        print(item)
      return res
    else:
      print('Empty result.')
      return None
    
def mysql_ping(host, user, password, db):
  with MySQLdb.connect(host, user, password, db, connect_timeout = 3306) as conn:
    conn: MySQLdb.Connection 
    conn.set_character_set('utf8')
    return conn.ping()

def mysql_connect(host, user, password, db):
  conn: MySQLdb.Connection = MySQLdb.connect(host, user, password, db, connect_timeout = 3306)
  return conn

def mysql_callback(host, user, password, db, callback):
  with MySQLdb.connect(host, user, password, db, connect_timeout = 3306) as conn:
    conn: MySQLdb.Connection 
    conn.autocommit(True)
    callback(conn)

def mysql_call(mysql: MySQLdb.Connection):
  status = mysql.stat()
  if status.startswith("Lost"):
    print('Reconnecting.')
    mysql.ping(True)
    print(mysql.stat())
  else:
    print(status) 

  print(f"Auto commit: {mysql.get_autocommit()}")

if Operation == Ping:
  mysql_ping(MySQL_Host, MySQL_Username, MySQL_Password, MySQL_DB)

elif Operation == Connect:
  mysql: MySQLdb.Connection = mysql_connect(MySQL_Host, MySQL_Username, MySQL_Password, MySQL_DB)
  mysql.ping(True)

elif Operation == Query and Input:
  mysql_query(MySQL_Host, MySQL_Username, MySQL_Password, MySQL_DB, Input)

elif Operation == Callback and Input:
  mysql_callback(MySQL_Host, MySQL_Username, MySQL_Password, MySQL_DB, Input)

elif Operation == CurlServer:
  !curl emasri.com &> /dev/null



In [ ]:
#@title ###### Data Template


In [ ]:
#@title ###### Callback
 
def mysql_call(conn: MySQLdb.Connection):
  pass


In [ ]:
#@title ##### XML

#Edit prices "round((x + x*0.65)/18.6, 2)"
#Edit currency "'USD'"

Callback = 10
Edit = 20
View = 30

Operation = Edit #@param {type: "raw"} ["Callback","Edit", "View"]
File = "/content/yenitoptanci1.xml" #@param {type: "string"}
Element = "IndirimliFiyat" #@param {type: "string"}
Input = None #@param {type: "raw"}
Attr = "" #@param {type: "string"}
Numbers = True #@param {type:'boolean'}

if File and Element:
  if Operation == Callback:
    data = xmlcall(File, Element, Input)

  elif Operation == Edit:
    xmledit(File, Element, value = Input, numbers = Numbers)

  elif Operation == View:
    stuff = xmlview(File, Element, attr = Attr)    
    
#with prop b_name = xml_data.find('child', {'name':'Frank'}) prop_value = b_name.get('test')

In [ ]:
#@title #####  WP API

import requests 
from requests.auth import HTTPBasicAuth

get = requests.get
post = requests.post

username = 'abbas'
password = 's$uCaoGGDL3Eai))BFEx9W)k'

In [ ]:
#@title ##### Set Variables

base = 'https://www.ghirasalkhaeer.com/wp-json'
media = '/wp/v2/media'
posts = '/wp/v2/posts'

if not 'json_obj'in globals():
  json_obj = ''
  obj = ''

In [ ]:
#@title ##### Posts
url = base + posts 
params = "&_fields=author,id,featured_media"

for obj in json_obj['posts']:
  data = {
      'format': 'standard',
      'date': obj['date'],
      'date_gmt': obj['dategmt'],
      'status': 'publish',
      'title': obj['title'],
      'content': obj['content'],
      'ping_status':'open',
      'categories': obj['categories'],
      'featured_media': obj['img_id']
  }

  res = post(url, auth = HTTPBasicAuth(username, password), data = data)

  if res:
    print( res.status_code)

In [ ]:
#@title ###### Product Prices

View = 0
Increase = 10
Decrease = 20
CurrencyChange = 30
CurrencyUpdate = 40

Operation = Increase #@param {type: "raw"} ["View","Increase","Decrease", "CurrencyChange", "CurrencyUpdate"]
File = "/content/yenitoptanci7.xml" #@param {type: "string"}
Price = "SatisFiyati" #@param {type: "string"}
SalePrice = "IndirimliFiyat" #@param {type: "string"}
Currency = "ParaBirimiKodu" #@param {type: "string"}
CurrencyUnit = "USD" #@param {type: "string"}
CurrencyRate = "0.054" #@param {type: "string"}
PriceChange = "65%" #@param {type: "string"}

def wcprices(file, price, saleprice, 
             pricechange = "",
             curr = "",
             currunit = "",
             currrate = "",
             op = ""):
  if not(pricechange) or op == View:
    if price:
      xmledit(file, price, value = "", numbers = True)
    
    if saleprice:
      xmledit(file, saleprice, value = "", numbers = True)

    if curr:
      xmledit(file, curr, value = "", numbers = False)
    return

  totalop = ''
  priceop = (op == Increase or op == Decrease or op == CurrencyChange)
  currunitupdate = ((curr and currunit) and (priceop or Operation == CurrencyUpdate))
  if priceop:
    if (price and pricechange) or currrate:
      sign = '+' if op == Increase else '-' 
      amount = f"x*{float(pricechange.replace('%',''))/100}" if '%' in pricechange else pricechange
      change = sign + amount if amount else ""
      totalchange = f"x{change}" if not currrate else f"(x{change})*{currrate}"
      print(f"{'Requested change:':>18} {totalchange}")
      totalop = f"round(({totalchange}), 2)"

  if currunitupdate:
    print(f"{'New currency:':>18} {currunit}")
  print()
  
  if totalop:
    if price:
      xmledit(file, price, value = totalop, numbers = True)
    
    if saleprice:
      xmledit(file, saleprice, value = totalop, numbers = True)    

  if currunitupdate:
    xmledit(file, curr, value = f"'{currunit}'", numbers = False)          
    
if File:
  wcprices(File, Price, SalePrice, 
            pricechange = PriceChange, 
            curr = Currency,
            currunit = CurrencyUnit,
            currrate = CurrencyRate,
            op = Operation)


In [ ]:
#@title ###### Map Products

File = "/content/yenitoptanci7.xml" #@param {type:'string'} 
Node = "Urun" #@param {type:'string'}

wocommerce_header = [
    'ID',
    'Type',
    'SKU',
    'Name',
    'Published',
    'Is featured?',
    'Visibility in catalog',
    'Short description',
    'Description',
    'Date sale price starts',
    'Date sale price ends',
    'Tax status',
    'Tax class',
    'In stock?',
    'Stock',
    'Backorders allowed?',
    'Sold individually?',
    'Weight (kg)',
    'Length (cm)',
    'Width (cm)',
    'Height (cm)',
    'Allow customer reviews?',
    'Purchase note',
    'Sale price',
    'Regular price',
    'Categories',
    'Tags',
    'Shipping class',
    'Images',
    'Download limit',
    'Download expiry days',
    'Parent',
    'Grouped products',
    'Upsells',
    'Cross-sells',
    'External URL',
    'Button text',
    'Position',
    'Attribute 1 name',
    'Attribute 1 value(s)',
    'Attribute 1 visible',
    'Attribute 1 global',
    'Attribute 2 name',
    'Attribute 2 value(s)',
    'Attribute 2 visible',
    'Attribute 2 global',
    'Meta: _wpcom_is_markdown',
    'Download 1 name',
    'Download 1 URL',
    'Download 2 name',
    'Download 2 URL']

defaultvalues = {
"ID":  "",
"Type":  "",
"SKU":  "",
"Name":  "",
"Published":  "1",
"Is featured?":  "1",
"Visibility in catalog":  "visible",
"Short description":  "",
"Description":  "",
"Date sale price starts":  "",
"Date sale price ends":  "",
"Tax status":  "taxable",
"Tax class":  "",
"In stock?":  "1",
"Stock":  "",
"Backorders allowed?":  "0",
"Sold individually?":  "0",
"Weight (kg)":  "",
"Length (cm)":  "",
"Width (cm)":  "",
"Height (cm)":  "",
"Allow customer reviews?":  "1",
"Purchase note":  "",
"Sale price":  "",
"Regular price":  "",
"Categories":  "",
"Tags":  "",
"Shipping class":  "class10",
"Images":  "",
"Download limit":  "",
"Download expiry days":  "",
"Parent":  "",
"Grouped products":  "",
"Upsells":  "",
"Cross-sells":  "",
"External URL":  "",
"Button text":  "",
"Position":  "0",
"Download 1 name":  "",
"Download 1 URL":  "",
"Download 2 name":  "",
"Download 2 URL":  ""
}

class keymapping:
  Id  = "" 
  SKU = "" 
  Name = "" 
  Featured = "" 
  Visibile = "" 
  ShortDescription = "" 
  Description = "" 
  Stock = "" 
  Weight = "" 
  Length = "" 
  Width = "" 
  Height = "" 
  Note = "" 
  SalePrice = "" 
  RegularPrice = "" 
  Category = "" 
  CategorySeparator = "" 
  Tags = "" 
  ShippingClass = ""
  Image = "" 
  Option = ""
  Attribute = ""
  AttributeName = "" 
  AttributeValue = "" 
  AttributeVisible = "" 

mapping = keymapping()
mapping.Id  = "VaryasyonID" #@param {type:'string'} 
mapping.SKU = "StokKodu" #@param {type:'string'} 
mapping.Name = "UrunAdi" #@param {type:'string'} 
mapping.Featured = "" #@param {type:'string'}
mapping.Visible = "" #@param {type:'string'} 
mapping.ShortDescription = "OnYazi" #@param {type:'string'}
mapping.Description = "Aciklama" #@param {type:'string'}
mapping.Stock = "StokAdedi" #@param {type:'string'} 
mapping.Weight = "Desi" #@param {type:'string'} 
mapping.Length = "" #@param {type:'string'} 
mapping.Width = "" #@param {type:'string'} 
mapping.Height = "" #@param {type:'string'} 
mapping.Note = "" #@param {type:'string'} 
mapping.SalePrice = "IndirimliFiyat" #@param {type:'string'} 
mapping.RegularPrice = "SatisFiyati" #@param {type:'string'} 
mapping.Category = "KategoriTree" #@param {type:'string'}
mapping.CategorySeparator = "/" #@param {type:'string'} 
mapping.Tags = "" #@param {type:'string'}
mapping.ShippingClass = "" #@param {type:'string'}  
mapping.Image = "Resim" #@param {type:'string'} 
mapping.Option = "Secenek" #@param {type:'string'} 
mapping.Attribute = "Ozellik" #@param {type:'string'} 
mapping.AttributeName = "Tanim" #@param {type:'string'} 
mapping.AttributeValue = "Deger" #@param {type:'string'} 
mapping.AttributeVisible = "" #@param {type:'string'} 

def wccats(values, separator):
  valuelist = values.split(separator)
  if valuelist:
    return slist(valuelist, sep = ' > ')

def wcmap(items, keymap: keymapping, defaultvalues: dict ):
  print(f"All items: {len(items)}\n")
  if keymap :
    export = []
    iattr = lambda item: itext(item, keymap.AttributeName)
    ival = lambda item: itext(item, keymap.AttributeValue)
    for item in items:
      options = item.find_all(keymap.Option)
      variable = len(options) > 1

      name = item.find(keymap.Name).text
      desc = item.find(keymap.Description).text
      shortdesc = itext(item.find(keymap.ShortDescription)) if keymap.ShortDescription else ""
      catsraw = item.find(keymap.Category).text
      cats = wccats(catsraw, keymap.CategorySeparator)
      imgitems = item.find_all(keymap.Image)
      imgs = list(map(itext, imgitems))
      imgstr = slist(imgs)

      if variable:
        parent = f"{options[0].find(keymap.SKU).text[:-1]}00"
        parentattrs = {}
        mapparent = defaultvalues.copy()
        mapparent["Type"] = 'variable'
        mapparent["SKU"] = parent
        mapparent["Name"] = name
        mapparent["Images"] = imgstr
        mapparent["Description"] = desc
        mapparent["Short description"] = shortdesc
        mapparent['Categories'] = cats

        if keymap.Tags:
          mapparent["Tags"] = itext(item.find(keymap.Tags))

        export.append(mapparent)
        print(f"Variable:   ->  {parent}")

      for option in options:
        mapw = defaultvalues.copy()

        if variable:
          mapw["Type"] = 'variation'
          mapw["Parent"] = parent

          attrset = option.find_all(keymap.Attribute)
          j = 1
          nameattr = ""
          for attr in attrset:
            attrname = attr.get(keymap.AttributeName)
            attrval = attr.get(keymap.AttributeValue)
            print(f"Variation:  ->  attr: {attrname}  val: {attrval}")
            mapw[f"Attribute {j} name"] = attrname
            mapw[f"Attribute {j} value(s)"] = attrval
            mapw[f"Attribute {j} global"] = "1"

            if not attrname in parentattrs:
              parentattrs[attrname] = set()
              parentattrs[attrname].add(attrval)
            else:
               parentattrs[attrname].add(attrval)
            if j == 1:
              nameattr = attrval 
          mapw["Name"] = f"{name} {nameattr}"

        else:
          mapw["Type"] = 'simple'
          mapw["Images"] = imgstr
          mapw["Short description"] = shortdesc
          
          mapw["Name"] = name
          mapw['Categories'] = cats
          mapw["Description"] = desc

          if keymap.Tags:
            mapw["Tags"] = itext(option.find(keymap.Tags))

        mapw["SKU"] = option.find(keymap.SKU).text
        mapw["Regular price"] = option.find(keymap.RegularPrice).text
        mapw["Sale price"] = itext(option.find(keymap.SalePrice)) if keymap.SalePrice else ""

        if keymap.Stock:
          stock = itext(option.find(keymap.Stock))
          mapw["Stock"] = stock
          mapw["In stock?"] = "0" if stock == "0" or stock == "" else "1"

        if keymap.Height:
          mapw["Height (cm)"] = itext(option.find(keymap.Height))           

        if keymap.Width:
          mapw["Width (cm)"] = itext(option.find(keymap.Width)) 

        if keymap.Length:
          mapw["Length (cm)"] = itext(option.find(keymap.Length)) 

        if keymap.Weight:
          mapw["Weight (kg)"] = itext(option.find(keymap.Weight))

        if keymap.ShippingClass:
          mapw["Shipping class"] = itext(option.find(keymap.ShippingClass))

        export.append(mapw)

      if variable:
        i = 1
        for attrkey in parentattrs:
          print(f"Attribute Values:  {attrkey}  ->  {str(parentattrs[attrkey])}\n")
          mapparent[f"Attribute {i} name"] = attrkey
          mapparent[f"Attribute {i} value(s)"] = slist(parentattrs[attrkey])
          mapparent[f"Attribute {i} visible"] = "1"
          mapparent[f"Attribute {i} global"] = "1"
      else:
        print(f"Simple:   ->  {mapw['SKU']}\n")

    return export

wcproc = lambda items: wcmap(items, mapping, defaultvalues)
importjob = xmlcall(File, Node, wcproc)

In [24]:
#@title ###### Products CVS

Minimal = csv.QUOTE_MINIMAL
All = csv.QUOTE_ALL
NonNumeric = csv.QUOTE_NONNUMERIC
_None = csv.QUOTE_NONE

FileName = "yenitoptanci7-1.csv" #@param {type:"string"}
Quoting = All # ["Minimal", "All", "NonNumeric", "_None"] {type:"raw"}

wcsv(wocommerce_header, importjob, FileName, quoting = Quoting)

In [ ]:
#@title ###### MySQL Functions
 
# Thumbnail id from another language translated with Ligotek

row = 'a:3:{s:8:"lingotek";a:4:{s:4:"type";s:4:"post";s:6:"source";i:3765;s:6:"status";s:7:"current";s:12:"translations";a:1:{s:5:"en_US";s:7:"current";}}s:2:"ar";i:3765;s:2:"en";i:4769;}'
postid_re = r'(?<="ar";i:)\d+'
query = "SELECT description FROM emasri_ghiras_prod.qtstkjza6_term_taxonomy where `description` like '%lingotek%';"

def mysql_lingotek(conn: MySQLdb.Connection):
  postid_re_ar = r'(?<="ar";i:)\d+'
  postid_re_en = r'(?<="en";i:)\d+'
  query = "SELECT description FROM emasri_ghiras_prod.qtstkjza6_term_taxonomy where `description` like '%lingotek%';"  
  print(f"Auto commit: {conn.get_autocommit()}")
  cur: MySQLdb.cursors.Cursor = conn.cursor()
  cur.execute(query)
  res = cur.fetchall()
  if res:
    for item in res:
      item = item[0]
      chk1 = re.search(postid_re_ar, item)
      chk2 = re.search(postid_re_en, item)
      if chk1 and chk2:
        postar = chk1.group()
        posten = chk1.group()
        imgquery = f"SELECT meta_value FROM emasri_ghiras_prod.qtstkjza6_postmeta where meta_key = '_thumbnail_id' and post_id = {postar};"
        cur.execute(imgquery)
        res2 = cur.fetchone()
        if res2:
          imgid = res2[0]
          query3 = f"insert into `emasri_ghiras_prod`.`qtstkjza6_postmeta`(`post_id`, `meta_key`, `meta_value`) VALUE ({posten}, '_thumbnail_id', {imgid});"
          cur.execute(query3)
        else:
          print("No cover id")
      else:
        print("No post id")  
##

# Create thumbnails from URLs

post_date = post_date_gmt = img_url = title = img_id = 0
"INSERT INTO qtstkjza6_posts (post_author, post_date, post_date_gmt, post_status, post_modified, post_modified_gmt, guid, post_type, post_mime_type) VALUES ( 77777,  '{post_date}' ,  '{post_date_gmt}' , 'inherit',  '{post_date}' ,  '{post_date_gmt}' ,  '{img_url}' , 'attachment', 'image/jpeg') ;"
last_id  = "SELECT LAST_INSERT_ID(); "
img_id = cur.execute(last_id)
"INSERT INTO `emasri_ghiras_prod`.`qtstkjza6_postmeta` (`post_id`, `meta_key`, `meta_value`) VALUES (  '{img_id}' , '_wp_attached_file', '{img_url}'); "
"INSERT INTO `emasri_ghiras_prod`.`qtstkjza6_postmeta` ( `post_id`, `meta_key`, `meta_value`) VALUES (  '{img_id}' , '_wp_attachment_image_alt', ''); "
"INSERT INTO `emasri_ghiras_prod`.`qtstkjza6_postmeta` ( `post_id`, `meta_key`, `meta_value`) VALUES (  '{img_id}' , '_wp_attachment_metadata', 'a:2:{s:5:\"width\";s:4:\"1620\";s:6:\"height\";s:4:\"1080\";}'); "

def mysql_thumbnails(conn: MySQLdb.Connection):
  print(f"Auto commit: {conn.get_autocommit()}")
  conn.set_character_set('utf8')
  cur: MySQLdb.cursors.Cursor = conn.cursor()

  for obj in json_obj['posts']:
    mysql_ensure(conn)

    post_date = obj['date']
    post_date_gmt = obj['dategmt']
    img_url = obj['image']
    title = obj['title']

    insert = f"INSERT INTO qtstkjza6_posts (post_author, post_date, post_date_gmt, post_status, post_modified, post_modified_gmt, guid, post_type, post_mime_type) VALUES ( 77777,  '{post_date}' ,  '{post_date_gmt}' , 'inherit',  '{post_date}' ,  '{post_date_gmt}' ,  '{img_url}' , 'attachment', 'image/jpeg') ;"
    cur.execute(insert)

    last_id  = "SELECT LAST_INSERT_ID(); "
    cur.execute(last_id)
    img_id = cur.fetchone()[0]
    obj['img_id'] = img_id

    insert = f"INSERT INTO `emasri_ghiras_prod`.`qtstkjza6_postmeta` (`post_id`, `meta_key`, `meta_value`) VALUES (  '{img_id}' , '_wp_attached_file', '{img_url}'); "
    cur.execute(insert)
    insert = f"INSERT INTO `emasri_ghiras_prod`.`qtstkjza6_postmeta` ( `post_id`, `meta_key`, `meta_value`) VALUES (  '{img_id}' , '_wp_attachment_image_alt', '{title}'); "
    cur.execute(insert)
    insert = f"INSERT INTO `emasri_ghiras_prod`.`qtstkjza6_postmeta` ( `post_id`, `meta_key`, `meta_value`) VALUES (  '{img_id}' , '_wp_attachment_metadata', 'a:2:{{s:5:\"width\";s:4:\"1620\";s:6:\"height\";s:4:\"1080\";}}'); "
    cur.execute(insert)
##




In [ ]:

#@title ##### Connection

display(IPython.display.Javascript("function ClickConnect() { console.info('Checking connection.'); btn = document.getElementById('ok'); if (btn != null) { console.info('Client is disconnected.'); console.log('Reconnecting.'); btn.click() } else{console.info('Client is connected.');} } setInterval(ClickConnect, 10000)"))


In [ ]:

#@markdown

!curl -s https://packagecloud.io/install/repositories/ookla/speedtest-cli/script.deb.sh | sudo bash &> /dev/null
!sudo apt-get install speedtest &> /dev/null

!speedtest

In [ ]:

#@markdown

ytimage = "https://img.youtube.com/vi/E429yLC6Riw/maxresdefault.jpg"
ytthumb = "https://img.youtube.com/vi/9SpTvpalmwc/hqdefault.jpg"


---


In [ ]:


#@markdown
# codex

import ftputil

# Download some files from the login directory.
with ftputil.FTPHost("ftp.domain.com", "user", "password") as ftphost:
  names = ftphost.listdir(ftphost.curdir)
  for name in names:
      if ftphost.path.isfile(name):
          ftphost.download(name, name)  # remote, local

  # Make a new directory and copy a remote file into it.
  ftphost.mkdir("newdir")
  with ftphost.open("index.html", "rb") as source:
      with ftphost.open("newdir/index.html", "wb") as target:
          ftphost.copyfileobj(source, target)  # similar to shutil.copyfileobj

  ftphost.upload(source, target, callback=None)
  ftphost.upload_if_newer(source, target, callback=None)


  ################


s0 = 0
if ftp.path.isfile(p0):
    s0 = ftp.path.getsize(p0)
else:  
    for path, dirs, files in ftp.walk(p0):
        for f in files:
            fp = path + '/' + f
            s0 += ftp.path.getsize(fp)
item = "{:>6}  {}".format(data_str(s0), name)
total_size += s0

In [ ]:

#@markdown

def write_json( dct, filename = 'scrape_output.json'):
  if not dct:
    error("Empty input")
    release()
    raise("Empty input") 
  
  text = json.dumps(dct, ensure_ascii = False, indent = 2)
  output_file = filename
  i = 0
  while True:
    i += 1
    if os.path.exists(output_file):
      sch = re.search(extdot_re, output_file)
      if sch:
        output_file = re.sub(extdot_re, f"{i}.", filename)
      else:
        output_file = f"{filename}{i}"  
    else: 
      break

  with open(output_file, 'w', encoding = 'utf8') as file:
    file.write(text)
    

In [ ]:
#@markdown

def xmlview(file, element, attr = ""):
  items = xmlcall(file, element)
  print(f"All items: {len(items)}\n")
  for i in range(len(items)):
    val = items[i].text if not attr else items[i].get(attr)
    print(f"Item {i:<5}->   {val:<10}")
  return items

In [ ]:
#@markdown

